#### Functions

In [5]:
def glue_tokens(tokens, order):
    """A useful way of glueing tokens together for
    Kneser Ney smoothing and other smoothing methods
    
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    return '{0}@{1}'.format(order,' '.join(tokens))

def unglue_tokens(tokenstring, order):
    """Ungluing tokens glued by the glue_tokens method"""
    if order == 1:
        return [tokenstring.split("@")[1].replace(" ","")]
    return tokenstring.split("@")[1].split(" ")

def tokenize_sentence(sentence, order):
    """Returns a list of tokens with the correct numbers of initial
    and end tags (this is meant ot be used with a non-backoff model!!!)
    
    :sentence: a string of text
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    tokens = sentence.split()
    tokens = ['<s>'] * (order-1) + tokens + ['</s>']
    return tokens

def find_unigrams(filename, MDF): # ex 1
    dataSource = open(filename, 'r')
    unigrams = Counter()

    for line in dataSource:
        words = tokenize_sentence(line,1)
        for w in words:
            unigrams[w] += 1

    unigramToRemove = [k for k in unigrams if unigrams[k] < MDF] #Combind training data pass 1 and 2 into one pass.
    for w in unigramToRemove:
        unigrams["<unk/>"] += unigrams[w] #allow for variable MDF
        del unigrams[w]

    return unigrams

def readInAndUnkOOV(filename, vocabCount): # exercise 1
    dataSource = open(filename, 'r')
    dataSansOOV = [];
    for line in dataSource:
        words = tokenize_sentence(line,1)
        words[:] = ['<unk/>' if not(x in vocabCount) else x for x in words]
        dataSansOOV.append(words)
       
    return dataSansOOV
        
def find_bigrams(filename, vocab, MDF): # exercise 2. Vocab represented by Unigram
    dataSource = open(filename, 'r')
    dataSansOOV = [];
    for line in dataSource:
        words = line.split(" ")
        words[:] = ['<unk/>' if not(x in vocab) else x for x in words]
        dataSansOOV.append(' '.join(words)) #rejoins the words now Unkified        
    

    bigrams = Counter() # a counter for how many times a given bigram sequence w_i-1,w_i occurs
    bigram_context = Counter() # a counter for how many times each word is used as a context word w_i-1 (so will include the start symbol)
    delta = 1  # delta is order - 1
    for line in dataSansOOV:
        words = tokenize_sentence(line, 2)  # tokenize sentence with the order 2 as the parameter
        #print(words)
        for i in range(delta, len(words)):
            context = words[i-delta:i]
            target = words[i]
            ngram = context + [target]
            bigrams[glue_tokens(ngram, 2)] +=1
            bigram_context[glue_tokens(context, 1)] += 1
    print(len(bigrams.keys()), "different bigrams")
    print(len(bigram_context.keys()), "different bigram contexts (and unigrams) observed")

    return bigrams, bigram_context
        
def prob_bigram_MLE(ngram):
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i-1)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)]
    denominator = bigram_context[glue_tokens(ngram[:1], 1)]
    prob = numerator / denominator
    return prob

def readIn(filename): # execise 2 mostly Reads in a file and splits it.
    dataSource = open(filename, 'r')
    data = [];
    for line in dataSource:
        words = tokenize_sentence(line,1)
        data.append(words)
    

#### Exercise 1

In [ ]:
from collections import Counter
from math import log

fileTraining = 'switchboard_lm_train.txt'
fileTesting = 'switchboard_lm_test.txt'
fileHeldOut = 'switchboard_lm_heldout.txt'

unigrams = find_unigrams(fileTraining,2) #the key values of the unigram are also a vocab.

unigram_total = sum(unigrams.values());
print(len(unigrams), "Unique unigrams observed")
print("unigram total", unigram_total)

testData = readInAndUnkOOV(fileTesting, unigrams)

s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s 
for sent in testData:
    # get the unigram model based probability of each sentence
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for w in sent:
        #print(str(unigrams[w]) + " " + w)
        prob = unigrams[w]/unigram_total
        logprob = log(prob, 2)  # the log of the prob to base 2
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    #print(sent, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("Test corpus cross entropy", cross_entropy)
print("Test corpus perplexity", perplexity)


#### Exercise 2

In [6]:
from collections import Counter
from math import log

fileTraining = 'switchboard_lm_train.txt'
fileTesting = 'switchboard_lm_test.txt'
fileHeldOut = 'switchboard_lm_heldout.txt'

unigrams = find_unigrams(fileTraining,1) #used to contruct MDF 2 Vocab from training set.
bigrams, bigram_context = find_bigrams(fileTraining, unigrams,2) #trains a bigram

''' 
for x in bigrams: 
    print(x) # for checking the bigram functions!
    print(bigrams[x])
''' 

#add one smoothing implementation

testData = readIn(fileTesting)



194181 different bigrams
18076 different bigram contexts (and unigrams) observed


#### Exercise 3

#### Exercise 4